# Chapter 14

All the use cases we have explored so far follow a very simple flow: a transaction hits the smart contract, the parameter sent with the transaction and the storage are pushed as a pair onto the stack, the code of the contract manipulates the stack without or without the help of environment variables and a pair containing a list of transactions (that was until now empty) and the new storage is returned.

However, smart contracts on the Tezos blockchain are also able to interact with the blockchain itself. They can create and send new transactions, they can delegate their funds to bakers and they can even originate new contracts! This will be the subject of this chapter.

The interactions between your smart contract and the blockchain allow for the creation of rich and complex applications that can run multiple actions in the background while keeping the requirement for action from your users minimal. Let's start with one of the most important ones, the creation of new transactions!

## Creating and sending transactions

Until now, we have always returned an empty list of transactions at the end of our smart contracts. A list of transactions, may it be empty or not, is necessary to stop the execution of a smart contract. At the end of the contract, there are two kinds of transactions you may want to send to the network: an amount of tokens to send to an implicit account or a transaction to another smart contract.  
It is important to remember that whatever happens inside the contract code has no effect on the blockchain. Nothing will be changed in the smart contract or the rest of blockchain before the contract returns a list of operations and its new storage. It may seem like a limitation, for example, it makes it more difficult to use data from another smart contract inside your contract, but it is actually a necessary security feature: this self-contained environment guarantees that no intervention from outside the contract can modify its storage. Your code modifies the storage without any outside influence in a very deterministic way.

That being say, it is of course possible to interact with the Tezos blockchain from the smart contract, at the end of the execution. One of the most common interactions is sending a transactions. If you write a contract that holds people's funds, there should be a way to send them back to them! You could also write a contract that fetches some data from another contract (for example an oracle) and needs to send a transaction.

Let's check first how sending a transaction to an implicit account looks like:

In [7]:
PATCH SENDER "tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb" ;
PATCH BALANCE 200 ;

parameter mutez ;
storage unit ;
code {
    CAR ;
    DUP ;
    BALANCE ;
    IFCMPLT
        { FAIL }
        {
            SENDER ;
            CONTRACT unit ;
            IF_NONE
                { FAIL }
                {
                    SWAP ;
                    UNIT ;
                    TRANSFER_TOKENS ;
                    NIL operation ;
                    SWAP ;
                    CONS ;
                    UNIT ;
                    SWAP ;
                    PAIR
                } ;
        };
} ;

RUN %default 100 Unit ;

PATCH: set SENDER=tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb;
PATCH: set BALANCE=200;
parameter mutez;
storage unit;
code { CAR ; DUP ; BALANCE ; { { COMPARE ; LT } ; IF { { UNIT ; FAILWITH } } { SENDER ; CONTRACT unit ; IF_NONE { { UNIT ; FAILWITH } } { SWAP ; UNIT ; TRANSFER_TOKENS ; NIL operation ; SWAP ; CONS ; UNIT ; SWAP ; PAIR } } } };
RUN: use %default; drop all; push (100, Unit);
CAR: pop (100, Unit); push 100;
DUP: push 100;
BALANCE: push 200;
COMPARE: pop 200, 100; push 1;
LT: pop 1; push False;
IF: pop False;
SENDER: push tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb;
CONTRACT: pop tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb; skip check; push ('tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb%default',);
IF_NONE: pop ('tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb%default',); push tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb%default;
SWAP: pop tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb%default, 100; push tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb%default; push 100;
UNIT: push Unit;
TRANSFER_TOKENS: pop Unit, 100, tz1VSUr8wwNhLAzempoch5

value    type
-------  ------
Unit     unit
kind         target                                  amount  entrypoint    parameters
-----------  ------------------------------------  --------  ------------  ------------
transaction  tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb       100  default       Unit

This simple smart contract takes an amount of `mutez` as a parameter, checks if the balance is greater than the requested amount and sends the amount to the sender of the transaction.

In order to create a transaction, you need three elements in the following order on the stack:
1. A parameter to send with the value of type `contract`
2. An amount in `mutez`
3. A value of type `contract`

This can be a bit confusing because here, we don't want to send a transaction to a contract but to an implicit account. However, implicit accounts can be seen as codeless contract. Because they don't have any code, they also don't accept any parameter or more precisely, they accept a parameter of type `unit`. 

To send a transaction to an implicit account, we will first cast an `address` into a `contract`. We can use the **`CONTRACT`** instruction with a parameter of type `unit` (because implicit accounts don't have a parameter). Remember that this instruction returns a value of type `option` so we have to use **`IF_NONE`**/**`IF_SOME`** to extract the value. When this is ready, we can add the amount in `mutez` we want to send (`0` is also acceptable). Finally, we have to push a value of type `unit` on top of the previous two elements, for example with **`UNIT`**. Once the stack is ready, we can call **`TRANFER_TOKENS`**. Although the name can be misleading, this is the instruction that will create a new transaction. Don't forget that, at this point, we have merely created the transaction and it hasn't been sent yet. The transaction will only be sent after we put it inside a list of elements of type `operation` and end the execution of the contract.

Now, you understood the basics of sending a transaction from a smart contract, so let's see how we can do the same to send a transaction to another smart contract:

In [9]:
parameter (pair address int) ;
storage unit ;
code {
    CAR ;
    UNPAIR ;
    CONTRACT int ;
    IF_NONE
        { FAIL }
        {
            SWAP ;
            PUSH mutez 0 ;
            SWAP ;
            TRANSFER_TOKENS ;
            NIL operation ;
            SWAP ;
            CONS ;
            UNIT ;
            SWAP ;
            PAIR
        } ;
} ;

RUN %default (Pair "KT1FM76xKAcN8EQkZVfBmeQzjzwNtpzYVwK3" 4) Unit ;

parameter (pair address int);
storage unit;
code { CAR ; { DUP ; CAR ; DIP { CDR } } ; CONTRACT int ; IF_NONE { { UNIT ; FAILWITH } } { SWAP ; PUSH mutez 0 ; SWAP ; TRANSFER_TOKENS ; NIL operation ; SWAP ; CONS ; UNIT ; SWAP ; PAIR } };
RUN: use %default; drop all; push (('KT1FM76xKAcN8EQkZVfBmeQzjzwNtpzYVwK3', 4), Unit);
CAR: pop (('KT1FM76xKAcN8EQkZVfBmeQzjzwNtpzYVwK3', 4), Unit); push ('KT1FM76xKAcN8EQkZVfBmeQzjzwNtpzYVwK3', 4);
DUP: push ('KT1FM76xKAcN8EQkZVfBmeQzjzwNtpzYVwK3', 4);
CAR: pop ('KT1FM76xKAcN8EQkZVfBmeQzjzwNtpzYVwK3', 4); push KT1FM76xKAcN8EQkZVfBmeQzjzwNtpzYVwK3;
DIP: protect 1 item(s);
CDR: pop ('KT1FM76xKAcN8EQkZVfBmeQzjzwNtpzYVwK3', 4); push 4;
restore 1 item(s);
CONTRACT: pop KT1FM76xKAcN8EQkZVfBmeQzjzwNtpzYVwK3; skip check; push ('KT1FM76xKAcN8EQkZVfBmeQzjzwNtpzYVwK3%default',);
IF_NONE: pop ('KT1FM76xKAcN8EQkZVfBmeQzjzwNtpzYVwK3%default',); push KT1FM76xKAcN8EQkZVfBmeQzjzwNtpzYVwK3%default;
SWAP: pop KT1FM76xKAcN8EQkZVfBmeQzjzwNtpzYVwK3%default, 4; push KT1FM76x

value    type
-------  ------
Unit     unit
kind         target                                  amount  entrypoint      parameters
-----------  ------------------------------------  --------  ------------  ------------
transaction  KT1FM76xKAcN8EQkZVfBmeQzjzwNtpzYVwK3         0  default                  4

This transaction is a little more straighforward that the previous one but follows exactly the same step, although with different values. We use an amount of `mutez 0` because we don't want to send any token to the contract but we replace the value of type `unit` with a value of type `int`. Note that although the address we send is a contract address, Michelson only sees an `address` and it must be cast to a `contract` value to process the transaction.